In [1]:
import pandas as pd
from scipy.stats import pearsonr

In [ ]:
%%time
wb=pd.read_csv('data/array/GSE132203_Geo_Submission_GTPEpic.csv',nrows=1000000,usecols=[0]+list(range(1,1591,2)),index_col=0).round(3)

In [ ]:
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE132nnn/GSE132203/matrix/GSE132203_series_matrix.txt.gz',skiprows=29+10,nrows=0,index_col=0).columns.str.strip('age: ').astype(float)

In [ ]:
%%time 
wb_t=wb.T
wb_t['age']=age
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

In [56]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/arc/hmr/neo',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [115]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/data/ref/EPIC.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')
man['bin']=man['cor']>0

In [173]:
cg=wb_cor[wb_cor>.057].head(1).index[0]
pearsonr(wb_t[cg],wb_t['age'])

PearsonRResult(statistic=0.05700003974028146, pvalue=0.10828807403539563)

In [174]:
man=man[man['cor']>.07].copy()

In [175]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append(-1)
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append(-1)
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            l.append(-1)
            cur+=1

CPU times: user 3.47 s, sys: 60.9 ms, total: 3.53 s
Wall time: 3.53 s


In [176]:
man['l']=l
mg=pd.DataFrame(man.groupby('l')[0].count())
mg=mg[mg[0]>=4].copy()
print(len(mg))
mg['cor']=man.groupby('l')['cor'].mean()
hmr.iloc[mg.sort_values('cor').tail(10).index][[0,1,2]]

1991


,0,1,2
3,,,
HYPO23585,chr19,37334236,37335116
HYPO41312,chr7,100176561,100177993
HYPO1919,chr1,47416145,47417392
HYPO1709,chr1,40708854,40709944
HYPO23941,chr19,44259825,44260516
HYPO24496,chr19,51887336,51888227
HYPO34661,chr4,48489542,48491631
HYPO19487,chr17,44353464,44354936
HYPO1989,chr1,52842501,52844181


In [177]:
%%time
man=man[man['l']>0].copy()
man['bin']=man['cor']>0
cont=[]
c=0
last=False
last_l=-1
for i,b in man[['l','bin']].iterrows():
    if (b['bin']):
        if (b['l']>last_l or last==False):
            c+=1
            last_l=b['l']
        cont.append(c)
        last=True  
    else:
        cont.append(-1)
        last=False

CPU times: user 1.56 s, sys: 24.9 ms, total: 1.59 s
Wall time: 1.59 s


In [178]:
man['cont']=cont
mgc=pd.DataFrame(man.groupby('cont')[0].count())
mgc['chr']=man.groupby('cont')[0].first()
mgc['b']=man.groupby('cont')[1].first()
mgc['e']=man.groupby('cont')[1].last()
mgc['cor']=man.groupby('cont')['cor'].mean()
topl=mgc[mgc[0]>=4].sort_values('cor')
topl.tail(50)

,0,chr,b,e,cor
cont,,,,,
2059,10,chr8,35235351,35236534,0.303085
760,6,chr3,120908063,120909443,0.304239
1408,12,chr5,135930103,135931206,0.304379
3895,5,chr19,56638098,56638261,0.305147
1191,15,chr5,16179100,16181423,0.305848
3574,5,chr18,55779979,55780609,0.306621
626,5,chr3,9552697,9553961,0.307475
863,8,chr3,180036744,180037446,0.309950
1862,16,chr7,28957539,28958978,0.311634


In [180]:
topl[topl['chr']=='chr2'].tail(10)

,0,chr,b,e,cor
cont,,,,,
499,4,chr2,179861600,179861969,0.261261
493,21,chr2,176187757,176189844,0.264868
440,9,chr2,119157791,119159109,0.269048
453,8,chr2,156320495,156323631,0.274451
415,5,chr2,73202662,73203517,0.275552
340,6,chr2,17878472,17880058,0.281213
428,7,chr2,74648135,74648433,0.281785
385,4,chr2,48755730,48755817,0.317170
553,5,chr2,209771814,209772119,0.326648
